In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from clda import ClassificationModel

In [12]:
import sys 

sys.path.append('../../../')

from bin.main import get_path 
from main import Manager 
from bin.utils.tools import encode_orders

m = Manager(get_path('../../../'))

2025-05-03 16:43:21,221 - bin.options.optgd.db_connect - INFO - Options DB Initialized


2025-05-03 16:43:21,224 - bin.options.optgd.db_connect - WARNING - You currently have 8 connections open.
2025-05-03 16:43:21,294 - INFO - PriceDB Initialized successfully at 2025-05-03 16:43:21.291777
2025-05-03 16:43:21,294 - bin.price.db_connect - INFO - PriceDB Initialized successfully at 2025-05-03 16:43:21.291777
2025-05-03 16:43:21,365 - INFO - Established 3 database connections
2025-05-03 16:43:21,365 - bin.price.db_connect - INFO - Established 3 database connections
2025-05-03 16:43:21,780 - bin.alerts.options_alerts - INFO - Notifications instance initialized.


In [13]:
data = m.Pricedb.model_preparation('spy', daily = True, start_date = "2022-12-31")

In [14]:
X = data['X']
y = data['y']
y = pd.Series(np.where(y > 0, 1, 0), name='target', index = X.index)
X_new = data['test_data'].to_frame().T
print(X.shape, y.shape, X_new.shape)

(553, 43) (553,) (1, 43)


In [15]:
# Initialize and run the model
model = ClassificationModel(
    X=X,
    y=y,
    numerical_cols=X.columns.tolist(),
    verbose = 0, 
    time_series=True
)

2025-05-03 16:43:23,432 - clda - INFO - ClassificationModel initialized with X shape: (553, 43)


In [16]:
model.preprocess_data()
model.train_models()


2025-05-03 16:43:23,801 - clda - INFO - Starting data preprocessing
2025-05-03 16:43:24,523 - clda - INFO - PCA data preparation completed with 3 components
2025-05-03 16:43:24,529 - clda - INFO - Data preprocessing completed. Training set shape: (387, 43)
2025-05-03 16:43:24,854 - clda - INFO - Evaluating LDA
2025-05-03 16:43:24,860 - clda - INFO - LDA evaluation completed. Accuracy: 0.596, F1 Score: 0.737
2025-05-03 16:43:25,060 - clda - INFO - Evaluating QDA
2025-05-03 16:43:25,317 - clda - INFO - QDA evaluation completed. Accuracy: 0.530, F1 Score: 0.680
2025-05-03 16:43:25,326 - clda - INFO - Evaluating Naive Bayes
2025-05-03 16:43:25,337 - clda - INFO - Naive Bayes evaluation completed. Accuracy: 0.566, F1 Score: 0.719
2025-05-03 16:43:25,835 - clda - INFO - Best KNN neighbors: 3
2025-05-03 16:43:25,836 - clda - INFO - Evaluating KNN
2025-05-03 16:43:25,861 - clda - INFO - KNN evaluation completed. Accuracy: 0.482, F1 Score: 0.590
2025-05-03 16:43:26,183 - clda - INFO - Best PCA 

In [17]:
# Display results
results = model.get_results()
print("\nModel Performance Results:")
print(results)



Model Performance Results:
                     Accuracy Precision  F1 Score Sensitivity Specificity
LDA                  0.596386  0.591195  0.737255    0.979167    0.071429
QDA                   0.53012  0.560811  0.680328    0.864583    0.071429
Naive Bayes          0.566265     0.575   0.71875    0.958333    0.028571
KNN                  0.481928   0.54386  0.590476    0.645833    0.257143
PCA KNN              0.542169   0.59434  0.623762     0.65625    0.385714
Logistic Regression  0.536145  0.568345   0.67234    0.822917    0.142857
Neural Network       0.518072  0.567797  0.626168    0.697917    0.271429


In [18]:
model.predict_new_data(X_new)

2025-05-03 16:43:28,957 - clda - INFO - Predicting on new data with shape: (1, 43)
2025-05-03 16:43:28,985 - clda - INFO - Predictions generated for 1 samples


,LDA,QDA,Naive Bayes,KNN,PCA KNN,Logistic Regression,Neural Network
2025-05-02,1,1,1,1,1,1,1


In [21]:
model.models.keys()

dict_keys(['LDA', 'QDA', 'Naive Bayes', 'KNN', 'PCA KNN', 'Logistic Regression', 'Neural Network'])

In [33]:
o = {}
for x in model.models.keys():
    preds = model.model_predictions[x]
    orders = encode_orders(predictions = preds.values, test_index=preds.index, stock = 'spy', shares = 10, name = x)
    o[x] = orders

orders = []; names = []
for x in o.keys():
    orders.append(o[x])
    names.append(x)

In [34]:
from backtest.strategies import Policy 

p = Policy(get_path('../../../'))

2025-05-03 16:48:15,384 - backtest.dbm - INFO - DBManager initialized with connections: dict_keys(['bonds_db', 'daily_db', 'intraday_db', 'ticker_path', 'inactive_db', 'backup_db', 'tracking_values_db', 'tracking_db', 'stats_db', 'vol_db', 'vol2_db', 'change_db', 'option_db', 'options_stat', 'dates_db', 'earnings_dict', 'stock_names', 'stock_info_dict', 'earnings_calendar', 'pics']) and pool size: 5
2025-05-03 16:48:15,386 - backtest.simulator - INFO - MarketSim initialized with verbose=False
2025-05-03 16:48:15,387 - backtest.dbm - INFO - DBManager object is being destroyed; closing all connections
2025-05-03 16:48:15,387 - backtest.dbm - INFO - All connections cleared


In [35]:
res = p.eval_multiple_orders(
    orders = orders,
    names = names, 
    sv = 10000, 
    commission = 1.0, 
    impact = 0.0005
)

2025-05-03 16:49:01,149 - backtest.dbm - INFO - Created new connection for database: daily_db
2025-05-03 16:49:01,217 - backtest.dbm - INFO - Created new connection for database: daily_db
2025-05-03 16:49:01,301 - backtest.simulator - INFO - Portfolio value computation completed, final value: $10,136.44
2025-05-03 16:49:01,303 - backtest.dbm - INFO - Created new connection for database: daily_db
2025-05-03 16:49:01,313 - backtest.dbm - INFO - Created new connection for database: daily_db
2025-05-03 16:49:01,716 - backtest.simulator - INFO - Portfolio value computation completed, final value: $10,111.96
2025-05-03 16:49:01,724 - backtest.dbm - INFO - Created new connection for database: daily_db
2025-05-03 16:49:01,739 - backtest.dbm - INFO - Created new connection for database: daily_db
2025-05-03 16:49:01,814 - backtest.simulator - INFO - Portfolio value computation completed, final value: $10,111.96
2025-05-03 16:49:01,816 - backtest.dbm - INFO - Created new connection for database: 

In [37]:
from bin.utils.tools import pretty_print

pretty_print(res)

,Stock,Days,StartDate,EndDate,StartBalance,EndBalance,Return,Commission,Impact
LDA,spy,166.00,2024-09-03,2025-05-01,"10,000.00","10,136.44",1.36,1.00,0.00
QDA,spy,166.00,2024-09-03,2025-05-01,"10,000.00","10,111.96",1.12,1.00,0.00
Naive Bayes,spy,166.00,2024-09-03,2025-05-01,"10,000.00","10,111.96",1.12,1.00,0.00
KNN,spy,166.00,2024-09-03,2025-05-01,"10,000.00","10,111.96",1.12,1.00,0.00
PCA KNN,spy,166.00,2024-09-03,2025-05-01,"10,000.00","10,228.12",2.28,1.00,0.00
Logistic Regression,spy,166.00,2024-09-03,2025-05-01,"10,000.00","10,136.44",1.36,1.00,0.00
Neural Network,spy,166.00,2024-09-03,2025-05-01,"10,000.00","10,111.96",1.12,1.00,0.00
Buy and Hold,spy,166.00,2024-09-03,2025-05-01,"10,000.00","10,208.25",2.08,0.00,0.00
Optimal Policy,spy,166.00,2024-09-03,2025-05-01,"10,000.00","14,069.04",40.69,0.00,0.00


In [39]:
m.Pricedb.ohlc('spy', daily = True, start = "2024-09-03")

,Open,High,Low,Close,Volume
Date,,,,,
2024-09-03,555.211691,555.548528,544.354556,546.900452,60600100.0
2024-09-04,545.038042,549.228337,544.304995,545.781006,47224900.0
2024-09-05,545.721572,548.604244,541.967091,544.453552,44264300.0
2024-09-06,544.780420,546.424819,534.378932,535.290283,68493800.0
2024-09-09,539.540072,542.571360,537.588523,541.283508,40445800.0
...,...,...,...,...,...
2025-04-28,551.390015,553.549988,545.020020,550.849976,47613800.0
2025-04-29,548.909973,555.450012,548.549988,554.320007,47775100.0
2025-04-30,547.570007,556.520020,541.520020,554.539978,93101500.0
